In [178]:
from pathlib import Path

import networkx as nx
from pyvis.network import Network
import numpy as np
from scipy import stats

The response variable is total population size after SIMULATION_YEARS.

In [179]:
NUM_WORLD_LOCATIONS = 100
SIMULATION_YEARS = 10000
INITIAL_POPULATION_PROPORTION = 10000 

In [180]:
world = Network(
    directed=True,
    neighborhood_highlight=True, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_world = nx.connected_watts_strogatz_graph(
    n=NUM_WORLD_LOCATIONS,
    k=5,
    p=0.5
)

In [181]:
for (v1, v2, weight) in pre_world.edges.data('weight'):
    # https://trenton3983.github.io/files/projects/2020-05-21_intro_to_network_analysis_in_python/2020-05-21_intro_to_network_analysis_in_python.html
    # https://stackoverflow.com/questions/40128692/networkx-how-to-add-weights-to-an-existing-g-edges

    # Here, the weights represent the ease of travelling between nodes.
    # A high weight indicates that travel is easy.
    pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=1)

# Make the graph directed to indicate
# allowable population movements.
pre_world = pre_world.to_directed()

In [182]:
# Skip the last row because we only care about
# the upper triangle exclusive of th main diagonal
# of the adjacency matrix.
for v1 in range(NUM_WORLD_LOCATIONS - 1):
    for v2 in range(v1 + 1, NUM_WORLD_LOCATIONS):
        current_edge_data = pre_world.get_edge_data(v1, v2)
        if current_edge_data is None:
            # There is no need to update current_weight.
            continue

        # Extract weight attribute
        current_weight = current_edge_data["weight"]
        if current_weight < 1:
            # Make the ease of travel different
            # for one of the edges connecting the same
            # pair of nodes to simulate ocean currents.
            pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=0.2)


In [183]:
betweenness_centralities = nx.betweenness_centrality(
    G=pre_world,
    weight="weight"
)

# Get a node with a maximal betweenness centrality.
# This node will hold our starting population.
# https://stackoverflow.com/a/280156/8423001
starting_node = max(betweenness_centralities, key=betweenness_centralities.get)

In [184]:
# Loop through nodes and set initial parameters.
for node in nx.nodes(G=pre_world):
    nx.set_node_attributes(
        G=pre_world, 
        values={node: {"carrying_capacity": 1000000}}
    )

In [185]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
world.show("world.html")